# Programação funcional

## Funções como objetos de primeira classe

[Wikipedia: First-Class_function](https://en.wikipedia.org/wiki/First-class_function)

Python trata funções como objetos de primeira classe. Ou seja: funções podem ser passadas como argumentos, retornadas como valor, e atribuidas a variáveis. Compare com outras linguagens:

- Java: funções não existem, apenas classes e objetos. Em Java, sempre que queremos trabalhar com o conceito de funções como objetos de primeira classe devemos simular este conceito através de classes. (Em Java 8 temos lambdas, mas estes são construídos através desta idéia de simular funções com classes.)

- C: Podemos simular funções de primeira classe através do uso de ponteiros para funções.

- Assembler: Que função? Só existe ```CALL``` e ```RET```!

Mas o que isso significa? Vamos entender na prática. Considere uma função que soma 1 ao valor passado e retorna este resultado.

In [ ]:
def soma_um(x):
    return x + 1


print(soma_um(12))

Podemos atribuir esta função a uma variável!

In [ ]:
minha_funcao = soma_um

print(minha_funcao(12))

Podemos também passar esta função como argumento de outras funções:

In [ ]:
def aplique_funcao(func, valor, repeticoes=1):
    print("O valor passado é {}".format(valor))
    for i in range(repeticoes):
        valor = func(valor)
        print(f"aplicando {i + 1} vez(es), o resultado é {valor}")
    print(f"O valor retornado será {valor}")
    return valor


aplique_funcao(soma_um, 12, 5)

Em Python podemos criar uma função em qualquer parte de nosso código.

In [ ]:
print("Antes de criar a função")


def triplica_valor(x):
    return 3 * x


print("Agora temos uma nova função.")
print(triplica_valor(11))

Podemos inclusive criar uma função dentro de uma função:

In [ ]:
import random


def my_sorting_function(vec):

    def my_inner_sorting_function(vec, inicio, fim):
        if inicio == fim:
            return

        def particiona(vec, inicio, fim):

            def troca(vec, i, j):  # NÃO É FUNÇÃO PURA! TEM EFEITOS COLATERAIS!
                aux = vec[i]
                vec[i] = vec[j]
                vec[j] = aux

            pivot = vec[inicio]
            meio = inicio + 1
            final_atual = inicio + 1

            while final_atual < fim:
                # Invariante:
                # vec[(inicio + 1) : meio] < pivot
                # vec[meio : final_atual] >= pivot

                # Se o novo elemento for menor que o pivot,
                # ele pertence à partição inferior.
                if vec[final_atual] < pivot:
                    troca(vec, final_atual, meio)
                    meio += 1

                final_atual += 1

            # Posiciona o pivot na sua posição correta.
            troca(vec, inicio, meio - 1)

            return meio - 1

        p = particiona(vec, inicio, fim)
        my_inner_sorting_function(vec, inicio, p)
        my_inner_sorting_function(vec, p + 1, fim)

    my_inner_sorting_function(vec, 0, len(vec))

In [ ]:
my_vec = [3, 1, 4, 2]
my_sorting_function(my_vec)
my_vec

In [ ]:
def test_sorting_function(
    sorting_function,
    num_tests=1000,
    vec_size=101,
    rand_range=100,
):

    def is_sorted(x):
        for i in range(len(x) - 1):
            if x[i] > x[i + 1]:
                return False
        return True

    for i in range(num_tests):
        vec = [random.randrange(rand_range) for _ in range(vec_size)]
        sorting_function(vec)
        if not is_sorted(vec):
            return False

    return True


print(test_sorting_function(my_sorting_function))

## Lambda

Podemos criar funções anônimas em Python, chamadas *lambda*. Lambdas são normalmente usados quando precisamos de uma função simples, que se resume a um *statement*. Veja este exemplo:

In [ ]:
def mapeia(func, vec):
    resultado = []
    for value in vec:
        resultado.append(func(value))
    return resultado


def quadrado(x):
    return x * x


dados = [2, 3, 5, 7]
res = mapeia(quadrado, dados)

print(dados)
print(res)

Note que a função ```quadrado()``` é muito simples. Nestes casos podemos usar um lambda:

In [ ]:
res2 = mapeia(lambda x: x * x, dados)
print(res2)

Veja como o lambda funciona: usamos a *keyword* ```lambda``` seguida dos argumentos da função e um dois-pontos. Em seguida vem uma expressão, que será avaliada e produzirá o valor retornado. Podemos usar mais de um argumento, como no exemplo abaixo:

In [ ]:
def reduz(func, vec, valor_inicial):
    valor = valor_inicial
    for item in vec:
        valor = func(item, valor)
    return valor


vec = list(range(10))
soma = reduz(lambda x, y: x + y, vec, 0)

print(vec)
print(soma)

Funções lambda não são essenciais à programação em Python, fica a seu cargo usar lambdas ou funções com nome. Mesmo o criador da linguagem Python (Guido van Rossum, o *"Benevolent Dictator For Life"* do Python) se arrepende de ter criado lambdas, e queria te-los removido do Python 3, mas a comunidade em geral acha lambdas úteis. Fica a seu cargo, não existe regra geral para adotar ou rejeitar o uso de lambdas.

### Exercício

Faça uma função ```filtra(func, vec)``` que recebe uma função ```func(valor)``` e uma lista ```vec```, e retorna uma lista com os valores de ```vec``` para os quais ```func(valor)``` é ```True```.

## Closures

https://www.programiz.com/python-programming/closure

Talvez você tenha percebido por acidente que em um escopo qualquer de Python temos acesso às variáveis do escopo e de escopos externos. Por exemplo:

In [ ]:
def func(value):
    return x + value  # x nem foi definido ainda!


x = 5
print(func(3))

x = 7
print(func(3))

Isso normalmente é fonte de complicações, e deve ser evitado a todo custo! Existe uma exceção: *closures*.

Em Python, *closure* é o nome dado à uma construção muito particular:

- Temos uma função definida dentro de uma função.
- Esta função interna usa variáveis do escopo da função externa.
- A função externa retorna a função interna.

Confuso? Vejamos um exemplo:

In [ ]:
# Eis um exemplo de closure
def aumentador(incremento):

    def _aumentador(x):
        return x + incremento

    return _aumentador


soma_um = aumentador(1)
soma_cinco = aumentador(5)
print(soma_um(3))
print(soma_cinco(3))

In [ ]:
# A partir de agora podemos até mesmo destruir aumentador()
del aumentador

# Veja que aumentador não existe mais:
try:
    aux = aumentador(3)
except NameError as e:
    print(e)

In [ ]:
# Porém soma_um() e soma_cinco() continuam firmes!
print(soma_um(5))
print(soma_cinco(5))

Em um closure o sistema faz um "backup" das variáveis do escopo externo, **mas apenas nas condições de closure**. Confira os atributos de ```soma_um``` (afinal funções são objetos):

In [ ]:
dir(soma_um)

Observe que temos um atributo "```__closure__```". Vejamos o que está contido neste atributo:

In [ ]:
type(soma_cinco.__closure__)

In [ ]:
len(soma_cinco.__closure__)

In [ ]:
soma_cinco.__closure__[0]

In [ ]:
soma_cinco.__closure__[0].cell_contents

Observe que o valor foi guardado no closure.

Ok, mas e daí? Para que servem os *closures*?

### Decorators

In [ ]:
# Eis o decorator
def temp_dec(nome):

    def imprime_args(func):

        def func_wrapper(*args, **kwargs):
            resultado = func(*args, **kwargs)
            print("Argumentos posicionais: {}".format(args))
            print("Argumentos nomeados: {}".format(kwargs))
            print("Comentário: {}".format(nome))
            print("Resultado: {}".format(resultado))
            return resultado

        return func_wrapper

    return imprime_args


# Aplicação do decorator
@temp_dec("Insper")
def uma_funcao_qualquer(x, y, z):
    return x + y + z


p = uma_funcao_qualquer(1, 2, z=3)
print(p)

### Exercício

Caching: faça um decorator para guardar um dicionário de valores já calculados de uma função. Se em uma nova chamada tivermos como argumento um valor já visto, retorna direto do dicionário, senão realmente chama a função sendo decorada.

Este padrão é tão comum que o Python já oferece esse decorator no pacote ```functools```. Veja este exemplo obtido da documentação da biblioteca:

In [ ]:
from functools import lru_cache


@lru_cache(maxsize=None)
def fib(n):
    if n < 2:
        return n
    return fib(n - 1) + fib(n - 2)


[fib(n) for n in range(16)]

In [ ]:
fib.cache_info()

### Aplicação parcial de argumentos

Eis um *closure* que recebe uma função que opera em dois argumentos (uma $f(x,y)$) e um valor $x_0$ e "congela" o primeiro valor, resultando em uma $g(y) = f(x_0,y)$".

In [ ]:
def aplica_primeiro_argumento(func, x):

    def wrapper(y):
        return func(x, y)

    return wrapper

Testando:

In [ ]:
def soma_dois_args(x, y):
    return x + y


print(soma_dois_args(2, 5))

In [ ]:
soma_cinco = aplica_primeiro_argumento(soma_dois_args, 5)
print(soma_cinco(2))

A aplicação parcial de argumentos é uma ferramenta tão comum em programação funcional que o Python tem uma função mais genérica para isso no pacote ```functools```:

In [ ]:
from functools import partial

soma_3 = partial(soma_dois_args, 3)
print(soma_3(10))

## List comprehensions

Antes de prosseguirmos na exploração de programação funcional em Python, vamos aprender sobre *list comprehensions*. Considere a função abaixo que retorna uma nova lista onde cada item é o quadrado do item equivalente na lista passada como argumento.

In [ ]:
vec = [2, 3, 5]

In [ ]:
def lista_quadrado(vec):
    res = []
    for item in vec:
        res.append(item**2)
    return res


vec_quad = lista_quadrado(vec)
print(vec_quad)

Vamos fazer o mesmo com *list comprehension*:

In [ ]:
vec_quad_2 = [x**2 for x in vec]
print(vec_quad_2)

Podemos também filtrar uma lista:

In [ ]:
vec = list(range(20))
vec_even = [x for x in vec if x % 2 == 0]
print(vec_even)

É possível, mas não muito recomendável, fazer multiplos `for` em um *list comprehension*:

In [ ]:
res = [(x, y) for x in [1, 2, 3] for y in ['a', 'b']]
res

Isto é equivalente a:

In [ ]:
res = []
for x in [1, 2, 3]:
    for y in ['a', 'b']:
        res.append((x, y))
res

Podemos inclusive estabelecer uma relação de dependência do segundo `for` em relação ao primeiro:

In [ ]:
res = [(i, j) for i in range(3) for j in range(i, 3)]
res

Isto é equivalente a:

In [ ]:
res = []
for i in range(3):
    for j in range(i, 3):
        res.append((i, j))
res

Podemos tambem incluir condicionais:

In [ ]:
res = [(i, j) for i in range(5) if i % 2 == 0 for j in range(i, 5)
       if j % 2 == 0]
res

Isto é equivalente à:

In [ ]:
res = []
for i in range(5):
    if i % 2 == 0:
        for j in range(i, 5):
            if j % 2 == 0:
                res.append((i, j))
res

## ```zip()```

Outra característica útil de Python é a função ```zip()```. Esta função recebe dois iteráveis e retorna um novo iterável formado por pares obtidos através da fusão elemento-a-elemento dos iteráveis iniciais. O iterável de pares que é retornado itera até que um dos iteráveis originais esteja esgotado. Por exemplo:

In [ ]:
x1 = [2, 3, 5]
x2 = ['abobora', 'batata', 'chuchu', 'tomate']

y = zip(x1, x2)
print(y)

In [ ]:
print(list(y))

In [ ]:
print([x for x in zip(x1, x2)])

In [ ]:
for primo, comida in zip(x1, x2):
    print(f'{comida}, {primo}')

### Exercício

Use ```sum()```, ```zip()``` e *list comprehension* para implementar o produto escalar de dois vetores em $\mathbb{R}^n$:

$$\left<x, y\right> = \sum_{i=1}^{n} x_i y_i$$

Voltamos agora à programação original.

## Funções puras

Uma função pura é uma função sem *efeitos colaterais* (*side-effects*): não altera a entrada, nem resulta em respostas diferentes para chamadas iguais.

Considere a função a seguir:

In [ ]:
def soma_um_em_tudo(vec):
    res = vec
    for i in range(len(res)):
        res[i] += 1
    return res


x = [1, 2, 3]
y = soma_um_em_tudo(x)

print(x)
print(y)

Observe que a chamada da função resultou na modificação do vetor de entrada! Esta é uma função com efeitos colaterais.

Considere agora esta outra versão:

In [ ]:
def soma_um_em_tudo(vec):
    res = vec.copy()
    for i in range(len(res)):
        res[i] += 1
    return res


x = [1, 2, 3]
y = soma_um_em_tudo(x)

print(x)
print(y)

Agora tudo bem, temos uma função pura.

Veja este outro exemplo:

In [ ]:
contador = 0


def foo(x):
    global contador
    contador += 1
    return x + contador


print(foo(1))
print(foo(1))

Este também é um exemplo de função com efeitos colaterais.

Funções puras são importantes porque:

- São mais fáceis de debugar;
- Podem ser cacheadas externamente;
- Podem ser paralelizáveis;
- Permitem demonstrar, em certos casos, que o sistema funciona matematicamente.

## Higher-order functions

Funções que recebem outras funções como argumento são chamadas de funções de ordem superior (*higher-order functions*). Algumas das mais importantes são ```map()```, ```filter()``` e ```reduce()```.

### ```map()```

A função ```map(func, iteravel)``` recebe uma função (preferencialmente pura) ```func``` e uma estrutura de dados iterável (como uma lista), e retorna um iterador onde cada elemento corresponde a um elemento do iterável inicial após aplicação da função. Por exemplo:

In [ ]:
vec = [2, 3, 5]
aux = map(lambda x: x**2, vec)
print(aux)

In [ ]:
resultado = list(aux)
print(resultado)

Note que poderíamos ter escrito um loop for para obter esse resultado. Com o map() não precisamos escrever loops (uma vantagem), e podemos deixar - em princípio - o Python decidir se quer rodar essa operação de mapeamento em paralelo, dividindo o trabalho entre vários *cores* (uma IMENSA vantagem!). 

### ```filter()```

A função ```filter(func, iteravel)``` recebe uma função ```func``` e um iterável e retorna um iterável cujos itens são aqueles onde ```func()``` retornou True ao ser aplicada aos itens do iteravel original. Por exemplo:

In [ ]:
vec = list(range(10))
aux = filter(lambda x: x % 2 == 1, vec)
print(aux)

In [ ]:
resultado = list(aux)
print(resultado)

### ```reduce()```
A função ```functools.reduce(func, iterable, initial)``` recebe uma função ```func(x,y)```, um iterável e um valor inicial, e combina os valores do iterável através da aplicação repetida de ```func()``` que serve para combinar os valores dois-a-dois, exatamente como na função ```reduz()``` acima. Se o valor inicial não for passado então o primeiro valor do iterável servirá para iniciar o processo. Por exemplo:

In [ ]:
from functools import reduce

vec = list(range(10))
soma = reduce(lambda x, y: x + y, vec)
print(soma)

### Exercício

Escreva um programa que calcula o produto do valor absoluto dos elementos de uma lista, para os valores não-nulos apenas. Não use loops: use map(), filter() e reduce().

Como você pode ver, ficou meio esquisito mas funciona. Vamos agora construir uma classe auxiliar que permite encadear as operações:

In [ ]:
class MeuDataFrameCaseiro:

    def __init__(self, dados):
        self.dados = dados

    def map(self, func):
        novos_dados = list(map(func, self.dados))
        return MeuDataFrameCaseiro(novos_dados)

    def filter(self, func):
        novos_dados = list(filter(func, self.dados))
        return MeuDataFrameCaseiro(novos_dados)

    def reduce(self, func, init):
        return reduce(func, self.dados, init)

Usando essa classe fica mais intuitivo escrever nossa tarefa acima:

In [ ]:
vec

In [ ]:
df = MeuDataFrameCaseiro(vec)

df \
    .filter(lambda x: x > 0) \
    .map(lambda x: x**2) \
    .reduce(lambda x, y: x + y, 0)

### Exercício

O código abaixo le as linhas de um arquivo CSV contendo dados sobre pacientes e se estes apareceram nas suas consultas médicas agendadas ou se faltaram. Escreva um programa que calcula a média de idade dos homens que faltaram às consultas. Não use loops, use apenas programação funcional.

In [ ]:
filename = 'pacientes.csv'

with open(filename, 'r', encoding='utf-8') as f:
    name_row = f.readline().strip().split(',')
    data = [x.strip().split(',') for x in f]

for i, name in enumerate(name_row):
    print(f'{i}: {name}')

In [ ]:
print(data[0])